**Import Some Libraries**

In [120]:
import math as m
import numpy as np # might not need this one. It has not been used yet.
from tabulate import tabulate

**Function Definitions**

In [121]:
def sqrt(number): # yeah, I am extremely lazy. What of it!? This basically makes it so I don't have to type "m" before every sqrt function
    return m.sqrt(number)

def find_a(fc,fy,As,b): # seems this is not in ACI 318-19 directly, but this is analysis per Whitney stress block using equilibrium where C=T where C = 0.85*fc*b*a and T = As*fy.
    a = (As*fy)/(0.85*fc*b)
    return a 

def find_beta1(fc): # from ACI 318-19 Table 22.2.2.4.3
    if fc <= 4:
        beta1 = 0.85
    elif fc >= 8:
        beta1 = 0.65
    else:
        beta1 = 0.85 - .05*(fc-4)
    return beta1

def find_c(a, beta1): # this is the depth to the neutral axis from using eqn 22.2.2.3 assuming "a" is known 
    c = a/beta1         
    return c

def find_Mn(fc,fy,As,b,d): # using Whitney stress block approach to finding nominal bending moment capacity
    a = find_a(fc,fy,As,b)
    Mn = As*fy*(d-a/2)
    return Mn

def find_ey(fy,Es): # ACI 318-19 20.2.2.1
    if fy == 60:
        ey = 0.002
    else:
        ey = fy/Es
    return ey

def find_Ec(fc,wc=None): # (19.2.2.1.a) or (19.2.2.1.b) # wc is density of concrete in lb/ft^3, Es is calculated in ksi for unit consistency. 
    if wc == None: # If wc not given, then normal weight concrete assumed (default function behavior)
        return 57000*sqrt(fc*1000)/1000 # use (19.2.2.1.b) for normal weight concrete
    if 60 < wc < 160:
        return wc**(1.5)*33*sqrt(fc*1000)/1000 # if wc is given, then (19.2.2.1.a) is used
    if wc < 60:
        raise Exception(f"{wc} lbs/cuft concrete is not dense enough (19.2.2.1)") # also watch depth/span ration per 7.3.1.1 and/or 9.3.1.1
    if wc > 160:
        raise Exception(f"{wc} lbs/cuft concrete is too dense (19.2.2.1)")

### use this function if lightweight concrete is being used. Not used by default
def find_lambda_conc(wc): # Table 19.2.4.1 (a)
    if wc <= 100:
        lam_c = 0.75
    if wc > 100: # Table 19.2.4.1 (a) eqn (b) and eqn (c) are covered here. For wc > 100, we use eqn (b), then...
        lam_c = 0.0075*wc
        if lam_c > 1.0: # ... if lambda_conc is greater than 1 as a result, we set lambda_conc equal to 1.0
            lam_c = 1.0
    return lam_c

def find_rho(As,b,d):   # ACI 318-19 2.2
    rho = As/(b*d)
    return rho

def find_rho_max(fc,fy,beta1,ey, eu=.003):   # no reference directly in code. Should use a strain diagram to prove this is true
    rho_max = 0.85*beta1*fc/fy*(eu)/(eu+ey+.003)
    return rho_max

def find_rho_bal(fc,fy,beta1,ey, eu=.003):   # no reference directly in code. Should use a strain diagram to prove this is true
    rho_bal = 0.85*beta1*fc/fy*(eu)/(eu+ey)
    return rho_bal

def find_et_min(ey):           
    et_min = ey + .003
    return et_min

def find_et(d,c):               # from using similar triangle relationship between c/eu = (d-c)/es    
    et_trans = .003*(d - c)/c           # note: net tensile strain at nominal capacity.
    return et_trans
      

def find_rho_min(fc,fy,d,h):    # 7.5.3.1 and 9.6.1.2 and 24.4.3.2 (also in 8.6.11 for two way slabs, but that is outside of this program's scope)
    rho_min = 3*sqrt(fc*1000)/fy/1000 # Using 9.6.1.2 # Note: 9.6.1.2 is technically for As,min but easily adjusted for rho by dividing As with b*d to produce shown formulas
    if rho_min < 200/(fy*1000):         ### Note: 9.6.1.1 and 9.6.1.2 need not be satisfied per 9.6.1.3 "If As,provided at every section is at least one-third greater than As,required by analysis"
        rho_min = 200/(fy*1000)         ### It seems that, if slab is wide enough, we do not need to limit rho min per 9.6.1 due to stress redistribution capable in slabs (leaving it this way to stay conservative for)
    if rho_min < .0018 * h / d: # As,temp = .0018 Ag = .0018 b h --> rho_temp = As,temp / (b*d) = .0018 b h /(b*d) = .0018 h /(d) ## seems this mainly pertains to slabs
        rho_min = .0018 * h /d  # per R7.6.1, technically reinforcement can be distributed to ea face, but R7.6.1 also recommends the reinforcement stay close as practical to tension face. 
        #If rho_min < rho_temp, then we make rho_temp be the new rho_min. Intent is to put tension steel closest to face. ## _*might be best to make a rho_temp and print it seperately*_
    return rho_min

def find_phi(rho,rho_max,rho_bal,et,ey): #  # Table 21.2.2 assuming no spiral reinforcement (note, since this is non-prestressed, ey = ety in this formula)
    if rho <= rho_max: #  Table 21.2.2 eqn (b)
        phi = 0.9
    elif rho_max < rho < rho_bal:      # 21.2.2 eqn (d)
        phi = .65 + .25*(et-ey)/.003
    else:                                # 21.2.2 eqn (f)
        phi = 0.65
    return phi

def comment_on_rho(rho,rho_min,rho_max): # FIND THE REFERENCE ## will not find one in code. They primarily use the strain relationships to do this.
    if rho < rho_min:
        print("You need to add more rebar!")
        print()
    elif rho > rho_max:
        print("You should remove some rebar or increase your beam, increse concrete strength or slab depth (𝜌 is greater than 𝜌_max)!")
        print()
    else:
        print("Reinforcement ratio, 𝜌, looks okay (𝜌 is greater than 𝜌_min and  \n (𝜌 is less or equal to 𝜌_max).")
        print()

def comment_on_phi(phi):    # FIND THE REFERENCE ## see 21.2.1 and 21.2.2
    if phi == 0.9:
        print(f"We're in the tension-controlled zone, which is good... phi = {phi:.2f} (for flexure). ... see 21.2.1 and 21.2.2.")
        print()
    elif 0.65 < phi < 0.9:
        print(f"Hmmm, we're in the transition zone... Check your design. phi = {phi:.2f}. ... see 21.2.1 and 21.2.2.")
        print()
    else:
        print(f"Yikes! We're over-reinforced! Sudden failure is possible if overloaded. \n ... Actually... please fix it. Don't do it. \n \
              Please! phi = {phi:.2f} (Very bad phi for flexure) ... see 21.2.1 and 21.2.2.")
        print()

def comment_on_h(d,h):  # this is just common sense. Reinforcement needs to be within "h" of the member
    if h < d:
        print("Check your dimensions, d cannot be larger than h! We're not in the Twilight Zone...")
        print()
    if h < d + 1.5:
        print("You might want to increase h to ensure proper concrete cover.")
        print()

def comment_on_lambda_conc(lambda_conc):
    if lambda_conc == 1:
        print("... Assuming normal weight concrete, so λ = 1.0 (19.2.4.3)")
        print()
    if lambda_conc != 1:
        print(f"... Assuming a concrete with λ = {lambda_conc:.1f} (19.2.4)")
        print()

def check_Nu(fc,b,h,Nu):
    if Nu > 0.10*fc*b*h:
        raise Exception(f"These calculations are not appropriate for the given member. {Nu} kip > {.1*fc*b*h} kip = 0.10*fc*Ag ...Member depth, h, or concrete strength, f'c, should be increased\
                        \n ... and/or calculations need to include stirrups and take into account axial loads per ACI 318-19 22.4 \
                        ... see also 9.5.2.2 and 6.2.5 for slenderness effects")




**Calculate Shear Capacity Values**

In [122]:
#Shear is complicated!

def find_Av_min(fc,fy,b): # ACI 318-19 Table 9.6.3.4 for nonprestressed members
    Av_min = .75*b/sqrt(fc*1000)/fy/1000
    if Av_min < 50*b/fy/1000:
        Av_min = 50*b/fy/1000   
    return Av_min

def find_lambda_size(d): # ACI 318-19 22.5.5.1.3
    if d <= 10:
        lambda_size = 1
    else:
        lambda_size = sqrt(2/(1+d/10))
    return lambda_size

def find_Vc(fc,b,d,h,As,Av,Av_min,member_type,lambda_size,lambda_conc): # ACI 318-19 7.6.3.1 or 9.6.3 and 22.5.5 (assuming no axial load to start)
    if member_type == "slab":
        if Av >= Av_min:
            print("Slab with Av >= Av_min (We have enough stirrups).")
            # 22.5.5.1 (a) chosen if Av >= Av_min
            Vc = 2*lambda_conc*sqrt(fc*1000)*b*d/1000                                     
            if Vc < 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000:
                # 22.5.5.1 (b) chosen if it is larger than (a)
                Vc = 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000                
        else:
            print("This is a slab with Av < Av_min (No stirrups or not enough stirrups).")
            # 22.5.5.1 (c) chosen if Av < Av_min                
            Vc = 8*lambda_size*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000   
    
    elif member_type == "beam":
        if Av >= Av_min:
            print("Beam with Av >= Av_min (We have enough stirrups).")
            Vc = 2*lambda_conc*sqrt(fc*1000)*b*d/1000                           # 22.5.5.1 (a)
            if Vc < 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000:
                Vc = 8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000       # 22.5.5.1 (b)
        else: # Av < Av_min
            print("This is a beam with Av < Av_min (No stirrups or not enough stirrups).")
            if h <= 10:
                print("... but h <= 10 inches, so we are not required to meet Av_min requirements per Table 9.6.3.1 to be able to use 22.5.5.1 (c).")    
                # use 22.5.5.1 (c) assuming rectangular beam with Av < Av_min and h <= 10 inches per Table 9.6.3.1                
                Vc = 8*lambda_size*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000
            else: # h > 10
                print("... and h > 10 inches, so we have to meet Av_min requirements unless we have enough Vc for Vu at critical section per 9.6.3 AND 22.5.5.1 (c).")
                Vc = 8*lambda_size*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000 # use 22.5.5.1 (c)
                # 9.6.3.1 limits shear strength for beams deeper than 10 inches, so if Vc from eqn 22.5.5.1 (c) is greater than the expression λ*sqrt(fc*1000)*b*d/1000
                if Vc > lambda_conc*sqrt(fc*1000)*b*d/1000:
                    Vc = lambda_conc*sqrt(fc*1000)*b*d/1000
                    print(" ... Might need some stirrups... 9.6.3.1 is limiting our shear strength!")
    print()
    return Vc

def adjust_Vc_with_Nu(Vc,fc,b,d,h,Nu): # ACI 318-19 22.5.5.1 and 22.5.5.1.2
    if Nu == 0:
        print("No axial load assumed, so no adjustment to Vc was done using Nu.")
        print()
    elif Nu != 0 and Nu/(6*b*h) <= .05*fc:
        Vc = Vc + Nu/6/(b*h)*b*d
        print(f"Axial load is present. Adjusting Vc using Nu = {Nu} kip since Nu/(6*b*h) = {Nu/(6*b*h):.1f} ksi is lower than 0.05fc = {.05*fc:.3f} ksi (See 22.5.5.1.2).")
        if Nu < 0:
            print("Interesting and unusual... Nu is negative, so there appears to be tension involved.\n  ...Anyways, that is going to hurt our shear strength a bit or a lot")
        print()
    else:
        Vc = Vc + .05*fc*b*d
        print(f"Axial load is present. Adjusting Vc by adding .05*fc*b*d = {.05*fc*b*d:.2f} kip since Nu is too high. (See 22.5.5.1.2)")
        print()
    return Vc

def check_max_Vc(Vc,fc,b,d,member_type,lambda_conc): # ACI 318-19
    if Vc > 5*lambda_conc*sqrt(fc*1000)*b*d/1000:  # ACI 318-19 22.5.5.1.1
        Vc = 5*lambda_conc*sqrt(fc*1000)*b*d/1000
        print(f"Vc is being capped by 22.5.5.1.1... The {member_type} might need to have a bigger cross-section!")
        print()
    return Vc  

**Input Values**

In [123]:
# Commonly not changed values
Es = 29000                  # ksi # ACI 318-19 20.2.2.2
wc = 145                    # lb/cuft # Note, only use this if concrete is known or if lightweight concrete per 19.2.4 
                            # (Do not use this weight directly in structural loading calculations... stick with 150 lbs/cuft!)

#Semi-Common inputs to change
fy = 60                     # ksi # Using grade 60 steel for the most part (fy = 60 ksi)
b = 12                      # inches # Not often changed for slabs and b=12 normally, as either E-width or effective b-width is used and loading is usual analyzed with *b = 12"* (per foot of width)

#Common inputs to change
fc = 5                      # ksi
h = 8                       # in
d = 6.13                    # in
As = .44*3                  # in^2
Av = 0                      # in^2 # this is stirrup As per space, s. Do not forget to include other leg of stirrup if, for example, U-stirrup
# note: should include calculations for spacing of stirrups required at some point
member_type = "beam"        # beam or slab # Note that if b-width is small in a portion of a slab, "beam" is the more conservative assumption especially regarding Vc
Nu = 47                     # kips, positive for compression, negative for tension


**Print Input Values**

In [124]:

print("___GIVEN___")
print(f"fy = {fy} ksi")
print(f"Es = {Es} ksi")
print(f"fc = {fc} ksi")
print(f"b = {b} in")
print(f"h = {h} in")
print(f"d = {d} in")
print(f"As = {As} in^2")
print(f"Av = {Av} in^2")
print(f"member_type = {member_type}")
print(f"Nu = {Nu} kips")
print(f"wc = {wc} lb/cuft")
print()

# Find values
beta1 = find_beta1(fc)  
lambda_conc = find_lambda_conc(wc)  # Using wc, the density of the concrete with Table 19.2.4.1(a) to find lambda. Most often this will be λ = 1.0 (normal weight concrete with wc > 135 lb/cuft)
lambda_size = find_lambda_size(d)   # from ACI 318-19 22.5.5.1.3 
Av_min = find_Av_min(fc,fy,b)       # ACI 318-19 Table 9.6.3.4 for nonprestressed members 
a = find_a(fc,fy,As,b)            
c = find_c(a, beta1)
et = find_et(d,c)
ey = find_ey(fy,Es)
et_min = find_et_min(ey)            
eu = 0.003
check_Nu(fc,b,h,Nu)                          # maximum strain at the extreme concrete compression fiber per 22.2.2.1 #Note: in 2.2 "Notation" this is εcu (epsilon_cu)


rho = find_rho(As,b,d)
rho_min = find_rho_min(fc,fy,d,h)
rho_max = find_rho_max(fc,fy,beta1,ey)
rho_bal = find_rho_bal(fc,fy,beta1,ey)
Mn = find_Mn(fc,fy,As,b,d)
phi = find_phi(rho,rho_max,rho_bal,et,ey)
phi_Mn = phi*Mn

print("___QUICK CHECKS___")
print()
print(f"rho = {rho:.4f}")
print(f"rho_min = {rho_min:.4f} <-- should be less than rho")
print(f"rho_max = {rho_max:.4f} <-- shall be greater than rho for tension controlled member (ideal)")
print(f"rho_bal = {rho_bal:.4f} <-- shall be greater than rho for member in transition (less ideal, avoid)")
print()
print(f"c = {c:.3f}")
print(f"ey = {ey:.4f}")
print(f"et = {et:.4f} <-- if this is higher than et_min, we are good (note this acts more like a 'gauge' for ductile concrete behaviour)")
print(f"et_min = {et_min:.4f}")
print(f"phi = {phi:.2f}")
print()
print(f"value = {lambda_conc*sqrt(fc*1000)*b*d/1000:.1f} ksi 9.6.3 reference for limiting beams without stirrups and with h > 10 in")
print(f"Vc per Table 22.5.5.1 (a) = {2*lambda_conc*sqrt(fc*1000)*b*d/1000:.1f} ksi") 
print(f"Vc per Table 22.5.5.1 (b) = {8*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000:.1f} ksi")
print(f"Vc per Table 22.5.5.1 (c) = {8*lambda_size*lambda_conc*(As/b/d)**(1/3)*sqrt(fc*1000)*b*d/1000:.1f} ksi") 
print(f"Vc_max per 22.5.5.1.1 = {5*lambda_conc*sqrt(fc*1000)*b*d/1000:.1f} ksi")

___GIVEN___
fy = 60 ksi
Es = 29000 ksi
fc = 5 ksi
b = 12 in
h = 8 in
d = 6.13 in
As = 1.32 in^2
Av = 0 in^2
member_type = beam
Nu = 47 kips
wc = 145 lb/cuft

___QUICK CHECKS___

rho = 0.0179
rho_min = 0.0035 <-- should be less than rho
rho_max = 0.0212 <-- shall be greater than rho for tension controlled member (ideal)
rho_bal = 0.0340 <-- shall be greater than rho for member in transition (less ideal, avoid)

c = 1.941
ey = 0.0020
et = 0.0065 <-- if this is higher than et_min, we are good (note this acts more like a 'gauge' for ductile concrete behaviour)
et_min = 0.0050
phi = 0.90

value = 5.2 ksi 9.6.3 reference for limiting beams without stirrups and with h > 10 in
Vc per Table 22.5.5.1 (a) = 10.4 ksi
Vc per Table 22.5.5.1 (b) = 10.9 ksi
Vc per Table 22.5.5.1 (c) = 10.9 ksi
Vc_max per 22.5.5.1.1 = 26.0 ksi


**Output Commentary and Do Some Shear Calcs**

In [125]:

print("___NOTES AND CHECKS___")
# Shear, Vc, is now wildly complicated to calculate, so checks are done during calculations for now, 
# which is why they are in this "checks" section.
print(f"Calculations being done through use of ACI 318 - 19 ... \n ... All code references are per this Code. \n")
Vc = find_Vc(fc,b,d,h,As,Av,Av_min,member_type,lambda_size,lambda_conc)
Vc = adjust_Vc_with_Nu(Vc,fc,b,d,h,Nu)
Vc = check_max_Vc(Vc,fc,b,d,member_type,lambda_conc)
phi_Vc = 0.75*Vc
comment_on_lambda_conc(lambda_conc)
comment_on_h(d,h)
comment_on_rho(rho,rho_min,rho_max)
comment_on_phi(phi)


___NOTES AND CHECKS___
Calculations being done through use of ACI 318 - 19 ... 
 ... All code references are per this Code. 

This is a beam with Av < Av_min (No stirrups or not enough stirrups).
... but h <= 10 inches, so we are not required to meet Av_min requirements per Table 9.6.3.1 to be able to use 22.5.5.1 (c).

Axial load is present. Adjusting Vc using Nu = 47 kip since Nu/(6*b*h) = 0.1 ksi is lower than 0.05fc = 0.250 ksi (See 22.5.5.1.2).

... Assuming normal weight concrete, so λ = 1.0 (19.2.4.3)

Reinforcement ratio, 𝜌, looks okay (𝜌 is greater than 𝜌_min and  
 (𝜌 is less or equal to 𝜌_max).

We're in the tension-controlled zone, which is good... phi = 0.90 (for flexure). ... see 21.2.1 and 21.2.2.



**Make Summary Table**

In [126]:

print("___CALCULATION TABLE___")

value_table_with_formulas = [
    ["fy (ksi)", fy, "ksi", "Given"],
    ["Es (ksi)", Es, "ksi", "Given"],
    ["f'c (ksi)", fc, "ksi", "Given"],
    ["b (in)", b, "inches", "Given"],
    ["d (in)", d, "inches", "Given"],
    ["As (in^2)", As, "sqin", "Given"],
    ["Av (in^2)", Av, "sqin", "Given"],
    ["...","","",""],
    ["Av_min (in^2)", f"{Av_min:.3f}", "sqin", "Table 9.6.3.4 for nonprestressed members"],
    ["𝜆s", f"{lambda_size:.3f}", "unitless", "(per 22.5.5.1.3)"],
    ["𝜆", f"{lambda_conc:.3f}", "unitless", "(per 22.5.5.1.3)"],
    ["et", f"{et:.5f}", "in/in", "from using similar triangle relationship between c/eu = (d-c)/es where es = et"],
    ["ey", f"{ey:.5f}", "in/in", "ey = fy/Es or 0.002 if fy = 60 ksi (per 21.2.2.1 and 21.2.2.2)"],
    ["eu",f"{eu:.5f}", "in/in", "maximum strain at the extreme concrete compression fiber (per 22.2.2.1) Note: Noted as εcu (per 2.2)"],
    ["𝛽1", f"{beta1:.2f}", "unitless", "Table 22.2.2.4.3"],
    ["a (in)", f"{a:.2f}", "inches", "a = As*fy/(0.85*fc*b) based on Whitney stress block distribution and allowance (R22.2.2.3 notes this is allowed)"],
    ["c (in)", f"{c:.2f}", "inches", "c = a/𝛽1 (per 22.2.2.4.1)"],
    
    ["𝜌", f"{rho:.4f}",         "sqin/in/in", "𝜌 = As/(b*d) (per 2.2)"],
    ["𝜌_min", f"{rho_min:.4f}", "sqin/in/in", "𝜌_min = 3*sqrt(fc*1000)/fy/1000 and less than 200/(fy*1000) ... also assures that rho_min is greater than (.0018 h / d)\n "],
    ["𝜌_max", f"{rho_max:.4f}", "sqin/in/in", "𝜌_max = 0.85*𝛽1 * fc/fy * (.003)/(eu + ey + 0.003) per setting 'et' to equal to 'ey + .003' like per R21.2.2b (finds max reinforcement ratio)"],
    ["𝜌_bal", f"{rho_bal:.4f}", "sqin/in/in", "𝜌_bal = 0.85*𝛽1 * fc/fy * (eu)/(eu+ey) per substitution using equalibrium for T=C"],
    
    ["...","","",""],
    ["𝜙", f"{phi:.2f}",   "factor", "Moment material reduction factor based on rho, rho_bal, rho_max,\n  and/or, if in transition zone, based on Table 21.2.2 (d)"],
    ["Mn (kip-in)", f"{Mn:.2f}",   "kip-in", "Mn = As*fy*(d-a/2)"],
    ["Vc (kip)", f"{Vc:.2f}",  "kip", "ACI 318-19 7.6.3.1 or 9.6.3 and 22.5.5 (assuming no axial load)"],
    ["...","","",""],
    ["𝜙Mn (kip-in)", f"{phi_Mn:.2f}", "kip-in", f"𝜙Mn = {phi:.2f}*{Mn:.2f} kip-in"],
    ["𝜙Mn (kip-ft)", f"{phi_Mn/12:.2f}",   "kip-ft", "𝜙Mn shown in kip-ft"],
    ["𝜙Vc (kip)", f"{phi_Vc:.2f}",   "kip", "𝜙Vc = 0.75*Vc"]
]

headers = ["Parameter", "Value", "Units", "Formula/Notes"]

print(tabulate(value_table_with_formulas, headers, tablefmt="markdown", numalign="right"))

___CALCULATION TABLE___
Parameter      Value    Units       Formula/Notes
-------------  -------  ----------  ---------------------------------------------------------------------------------------------------------------------------------------------
fy (ksi)       60       ksi         Given
Es (ksi)       29000    ksi         Given
f'c (ksi)      5        ksi         Given
b (in)         12       inches      Given
d (in)         6.13     inches      Given
As (in^2)      1.32     sqin        Given
Av (in^2)      0        sqin        Given
...
Av_min (in^2)  0.010    sqin        Table 9.6.3.4 for nonprestressed members
𝜆s             1.000    unitless    (per 22.5.5.1.3)
𝜆              1.000    unitless    (per 22.5.5.1.3)
et             0.00647  in/in       from using similar triangle relationship between c/eu = (d-c)/es where es = et
ey             0.00200  in/in       ey = fy/Es or 0.002 if fy = 60 ksi (per 21.2.2.1 and 21.2.2.2)
eu             0.00300  in/in       maximum strain at